# Food project

## Discovery (identificazione del problema e degli obiettivi)

Negli ultimi secoli la popolazione mondiale è aumentata notevolmente e secondo l'ONU (fonte del 2019), nel 2050 potrebbe arrivare a quota *10 miliardi*. 

In maniera direttamente proporzionale **sono aumentate anche le domande di cibo, energia ed acqua**, per soddisfare questo bisogno. 

Sfortunamente, i costanti monitoraggi sul cambiamento climatico suggeriscono che il clima della Terra sta cambiando pericolosamente e *parte di questo effetto è direttamente correlato alla produzione agricola e all'allevamento*:

- la produzione alimentare richiede acqua ed energia; 
- la produzione energetica tradizionale richiede risorse idriche; 
- l'agricoltura fornisce una potenziale fonte di energia e tanta acqua. 
 
Garantire che nel mondo tutte le persone abbiano accesso a una dieta nutriente *in modo sostenibile* è una delle maggiori sfide che dobbiamo affrontare. 


> **L'obiettivo di questo notebook** è quello di sfruttare alcuni dati inerenti alla *produzione e distribuzione mondiale di alimenti e mangimi*, per valutarne il loro impatto sull'ambiente in termini di *utilizzo di risorse idriche ed emissioni di gas serra*. 

### Principali spunti e domande che guideranno questa analisi:

1. Il confronto tra produzione di cibo e mangime a livello mondiale

2. Quali paesi sono i maggiori produttori di cibo e mangimi?

3. Quali sono i paesi in maggior crescita in termini di produzione di cibo/consumo? 

4. Sono presenti interessanti outlier nei dati per quanto riguarda la produzione?

5. Quali tipi di alimenti hanno un impatto maggiormente negativo sull'ambiente?

    - Confrontare l'impatto ecologico degli alimenti a base animale
    - Confrontare l'impatto ecologico degli alimenti a base vegetale
    - Confrontare l'impatto ecologico degli alimenti ricchi di proteine


6. Quale fase della produzione alimentare contribuisce maggiormente all'emissione di gas serra?

7. Quali tipi di produzione alimentare dovrebbero essere incoraggiati per il consumo di una dieta maggiormente sostenibile per l'ambiente?

---

## Data selection

Questo progetto sfrutta due datasets: 

- Il primo, fornito dalla FAO (Food and Agriculture Organization of the United Nations), mostra una panoramica della produzione alimentare mondiale dal 1961 al 2013, concentrandosi su un confronto tra alimenti prodotti per il consumo umano e mangimi prodotti per l'allevamento di animali. 
- Il secondo contiene i valori di utilizzo dell'acqua e le emissioni di gas serra necessari per la realizzazione dei 43 alimenti più comuni prodotti nel mondo.

Per entrambi i dataset, saranno selezionati in maniera specifica i valori utili alla realizzazione delle analisi indicate nella precedente sezione [Discovery](#principali-spunti-e-domande-che-guideranno-questa-analisi).

### Import di dati, moduli e funzioni utili

In [11]:
import pandas as pd
import numpy as np 
import os
import matplotlib.pyplot as plt
import seaborn as sns
from zipfile import ZipFile

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 63)

In [2]:
#Funzioni create da me :)
def zip_extractor():
    """This function looks for zip files and extracts them in the same file path of this script, or in a folder provided 
    with an input. If the folder doesn't exist, it will be created with the provided name."""
    
    destination = input('Where do you want to put the extracted files?\nPress enter if you want to extract files in the current path: ').capitalize()

    if destination == '':
        pass
    elif destination not in os.listdir():
        os.mkdir(destination)

    for dirname, _, filenames in os.walk(os.getcwd()):
        if dirname != 'Zip files':
            for filename in filenames:
                if '.zip' in filename:
                    with ZipFile(filename, "r") as zip:
                        zip.extractall(f'{destination}')

    print("Extraction: Done")
    
    
def csv_nan_reader():
    
    """This function search for .csv files in a target folder (provided by input), skipping 'checkpoint' files 
    and read them in pandas as DataFrame. Then for each file are printed out: 
    
    - filename and parental folder 
    - Dataframe shape 
    - number of total cells 
    - number of cells with missings 
    - % of missing data 
    - rows and columns containing missings 
    - rows and columns in original dataset 
    - rows and columns remained after drop 
    - the effect of dropping rows and columns in terms of remaining data and which is the best method.
    
    If a folder is empty or doesn't exist, this function will segnalate it."""
    
    target_folder = input('Write here folder name: ').capitalize()
    datano = 0

    # Check if the folder exist
    if os.path.isdir(target_folder):
        
        # Check if the folder is not empty
        if os.listdir(target_folder):
            
            for dirname, _, filenames in os.walk(target_folder):
                for filename in filenames:
                    if not 'checkpoint' in filename:
                        name, file_ext = os.path.splitext(filename)
                        
                        if '.csv' in file_ext:
                            
                            path = os.path.join(dirname, filename)
                            df = pd.read_csv(f'{path}')
                            
                            # Extract main information of the file and count Dataframes
                            datano += 1
                            print(f'\n\nDataframe No: {datano}')
                            print('_'*20+'Start'+'_'*20)
                            print(f'This is the Dataset "{filename}" from folder "{dirname}"')
                            print(filename,'shape',df.shape)
                            
                            # How many total missing values do we have?
                            total_cells = np.product(df.shape)
                            print('Total cells:', total_cells)
                            
                            total_missing = df.isnull().sum().sum()
                            print('Total cells with missings:', total_missing)
                            
                            # Count rows containing missing values and check what happens if you drop rows
                            rows_with_missing = df[df.isnull().any(axis=1)].index.to_list()
                            print('\nTotal lenght of missing rows:', len(rows_with_missing))
                            
                            if len(rows_with_missing) > 10:
                                print(f'Rows containing missing values:\n{rows_with_missing[:10]}...[{rows_with_missing[-1]}]')
                            else:
                                print('Rows containing missing values:\n',rows_with_missing)
                                
                            drop_rows = df.dropna()
                            rows_removal_perc = round((1-drop_rows.shape[0]/df.shape[0])*100,2)
                            print(f"\nRows in original dataset: {df.shape[0]}")
                            print(f"Rows remained after drop: {drop_rows.shape[0]}")
                            print('\nDropping rows with NaN removed',rows_removal_perc,'% of the data!')
                            print(f'Shape of Dataframe after rows manipulation: {drop_rows.shape}')
                            
                            # Count columns containing missing values and check what happens if you drop columns
                            cols_with_missing = [col for col in df.columns if df[col].isnull().any()]
                            print('\n\nTotal lenght of missing columns:', len(cols_with_missing))
                            
                            if len(cols_with_missing) > 5:
                                print(f'Columns containing missing values:\n{cols_with_missing[:5]}...[{cols_with_missing[-1]}]')                                
                            else:
                                print('Columns containing missing values:\n',cols_with_missing)
                                
                            drop_cols = df.dropna(axis=1)
                            cols_removal_perc = round((1-drop_cols.shape[1]/df.shape[1])*100,2)
                            print(f"\nColumns in original dataset: {df.shape[1]}")
                            print(f"Columns remained after drop: {drop_cols.shape[1]}")
                            print('\nDropping columns with NaN removed',cols_removal_perc,'% of the data!')
                            print(f'Shape of Dataframe after columns manipulation: {drop_cols.shape}\n\n')

                            # Show info for each col: dtype, unique, %missing
                            print("Dtype, Unique values and Missing(%) of each columns:\n")
                            df_info= pd.DataFrame({"Dtype": df.dtypes, "Unique": df.nunique(),
                            "Missing%": round(df.isnull().sum()/df.shape[0]*100, 2)})
                            print(df_info, '\n')
                            
                            # Compare rows drop vs cols drop and print what is better
                            if rows_removal_perc < cols_removal_perc:
                                print(f"Dropping rows with NaN is the best approach.({rows_removal_perc}% vs {cols_removal_perc}%)\n\n")
                            elif rows_removal_perc > cols_removal_perc:
                                print(f"Dropping columns with NaN is the best approach.({cols_removal_perc}% vs {rows_removal_perc}%)\n\n")
                            else:
                                print("Dropping rows or columns produced the same output.")
                            print('-'*20+'End'+'-'*20+'\n\n')
                            
                        else:
                            print(f"\nI found a file with different extension from csv. '{name}' is a '{file_ext}' file.")
            
            print('\nAll Done.')
        
        else:
            print('\nThis folder is empty!')
    else:
        print("\nSelected folder doesn't exist in this path!")

In [3]:
# !kaggle datasets download -d dorbicycle/world-foodfeed-production
# !kaggle datasets download -d selfvivek/environment-impact-of-food-production
# zip_extractor()

In [4]:
#Dataset per la produzione ci cibi/mangimi dai vari paesi del mondo
fao_df = pd.read_csv('Data/Fao.csv')
df_prod = fao_df.copy()

#Dataset relativo ai consumi di acqua e alla produzione di gas serra
consumption_df = pd.read_csv('Data/Food_Production.csv')
df_cons = consumption_df.copy()

## Data cleaning

Procedo verificando la presenza di valori nulli (*NaN*) nei dataset, o altri valori incoerenti. Se effettivaemente presenti, voglio sapere quanti sono e in quali colonne sono maggiormente frequenti. Inoltre, proverò a fare un *imputation*, ovvero provare a stabilire il motivo della loro presenza. 

La procedura prenderà in analisi un dataset alla volta. 

### Dataset per la produzione ci cibi/mangimi dai vari paesi del mondo

In [40]:
print(f'Dimensioni del dataframe: {df_prod.shape}')

# Quante celle contengono valori nulli rispetto alle celle totali?
total_cells = np.product(df_prod.shape)
print(f'Numero totale di celle: {total_cells}')

total_missing = df_prod.isnull().sum().sum()
print(f'Numero totale di celle con valori nulli: {total_missing}')

percent_missing = round(total_missing/total_cells * 100, 2)
print(f'\nPercentuale di valori nulli in questo Dataframe: {percent_missing}%')

Dimensioni del dataframe: (21477, 63)
Numero totale di celle: 1353051
Numero totale di celle con valori nulli: 117450

Percentuale di valori nulli in questo Dataframe: 8.68%


In [81]:
# Creo un dataframe per avere un riassunto di tutte le colonne
print("Dtype, Valori unici e Valori mancanti(%) di ciascuna colonna:")

df_prod_info= pd.DataFrame({ 
                            "Dtype": df_prod.dtypes.values, 
                            "Valori unici": df_prod.nunique(),
                            "Valori mancanti(%)": round(df_prod.isnull().sum()/df_prod.shape[0]*100, 2)})

df_prod_info

Dtype, Valori unici e Valori mancanti(%) di ciascuna colonna:


,Dtype,Valori unici,Valori mancanti(%)
Area Abbreviation,object,169,0.00
Area Code,int64,174,0.00
Area,object,174,0.00
Item Code,int64,117,0.00
Item,object,115,0.00
Element Code,int64,2,0.00
Element,object,2,0.00
Unit,object,1,0.00
latitude,float64,173,0.00
longitude,float64,174,0.00


In [56]:
# Count rows containing missing values and check what happens if you drop rows
rows_with_missing = df_prod[df_prod.isnull().any(axis=1)].index.to_list()
print('\nTotal lenght of missing rows:', len(rows_with_missing))

df_prod.loc[rows_with_missing].sample(10)


Total lenght of missing rows: 3539


,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
20661,UZB,235,Uzbekistan,2805,Rice (Milled Equivalent),5521,Feed,1000 tonnes,41.38,64.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1,2
11062,LVA,119,Latvia,2516,Oats,5142,Food,1000 tonnes,56.88,24.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0,5.0,18.0,18.0,16.0,19.0,18.0,15.0,16.0,15.0,16.0,8.0,10.0,8,10
12975,MNE,273,Montenegro,2515,Rye and products,5521,Feed,1000 tonnes,42.71,19.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,0
20724,UZB,235,Uzbekistan,2645,"Spices, Other",5142,Food,1000 tonnes,41.38,64.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1
17339,SVK,199,Slovakia,2543,"Sweeteners, Other",5142,Food,1000 tonnes,48.67,19.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,18.0,18.0,27.0,24.0,7.0,10.0,5.0,14.0,2.0,3.0,26.0,46.0,38.0,37.0,65.0,77.0,69.0,49.0,63,79
4997,HRV,98,Croatia,2761,Freshwater Fish,5142,Food,1000 tonnes,45.10,15.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,4.0,3.0,2.0,3.0,3.0,3.0,3.0,4.0,3.0,3.0,3.0,4.0,5.0,4.0,5.0,6.0,8.0,8.0,6,6
1181,AZE,52,Azerbaijan,2909,Sugar & Sweeteners,5142,Food,1000 tonnes,40.14,47.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.0,85.0,92.0,99.0,110.0,120.0,119.0,99.0,89.0,105.0,131.0,137.0,132.0,153.0,160.0,119.0,128.0,131.0,144.0,148.0,164,173
733,ARM,1,Armenia,2605,"Vegetables, Other",5142,Food,1000 tonnes,40.07,45.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224.0,174.0,195.0,213.0,240.0,212.0,232.0,285.0,220.0,257.0,318.0,379.0,402.0,478.0,510.0,632.0,621.0,636.0,562.0,603.0,715,724
14683,OMN,221,Oman,2740,"Butter, Ghee",5142,Food,1000 tonnes,21.51,55.92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,5.0,5.0,6.0,5.0,5.0,3.0,2.0,2.0,1.0,2.0,5.0,0.0,5.0,0.0,5.0,5.0,4.0,5.0,4.0,4.0,4.0,3,2
1089,AZE,52,Azerbaijan,2516,Oats,5142,Food,1000 tonnes,40.14,47.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1,1


In [7]:
drop_rows = df_prod.dropna()
rows_removal_perc = round((1-drop_rows.shape[0]/df_prod.shape[0])*100,2)
print(f"\nRows in original dataset: {df_prod.shape[0]}")
print(f"Rows remained after drop: {drop_rows.shape[0]}")
print('\nDropping rows with NaN removed',rows_removal_perc,'% of the data!')
print(f'Shape of Dataframe after rows manipulation: {drop_rows.shape}')

# Count columns containing missing values and check what happens if you drop columns
cols_with_missing = [col for col in df_prod.columns if df_prod[col].isnull().any()]
print('\n\nTotal lenght of missing columns:', len(cols_with_missing))

if len(cols_with_missing) > 5:
    print(f'Columns containing missing values:\n{cols_with_missing[:5]}...[{cols_with_missing[-1]}]')                                
else:
    print('Columns containing missing values:\n',cols_with_missing)
    
drop_cols = df_prod.dropna(axis=1)
cols_removal_perc = round((1-drop_cols.shape[1]/df_prod.shape[1])*100,2)
print(f"\nColumns in original dataset: {df_prod.shape[1]}")
print(f"Columns remained after drop: {drop_cols.shape[1]}")
print('\nDropping columns with NaN removed',cols_removal_perc,'% of the data!')
print(f'Shape of Dataframe after columns manipulation: {drop_cols.shape}')
# Show info for each col: dtype, unique, %missing
print("Dtype, Unique values and Missing(%) of each columns:\n")
df_prod_info= pd.DataFrame({"Dtype": df_prod.dtypes, "Unique": df_prod.nunique(),
"Missing%": round(df_prod.isnull().sum()/df_prod.shape[0]*100, 2)})
print(df_prod_info, '\n')

# Compare rows drop vs cols drop and print what is better
if rows_removal_perc < cols_removal_perc:
    print(f"Dropping rows with NaN is the best approach.({rows_removal_perc}% vs {cols_removal_perc}%)\n\n")
elif rows_removal_perc > cols_removal_perc:
    print(f"Dropping columns with NaN is the best approach.({cols_removal_perc}% vs {rows_removal_perc}%)\n\n")
else:
    print("Dropping rows or columns produced the same output.")
print('-'*20+'End'+'-'*20+'\n\n')


df_prod.head()


Rows in original dataset: 21477
Rows remained after drop: 17938

Dropping rows with NaN removed 16.48 % of the data!
Shape of Dataframe after rows manipulation: (17938, 63)


Total lenght of missing columns: 51
Columns containing missing values:
['Y1961', 'Y1962', 'Y1963', 'Y1964', 'Y1965']...[Y2011]

Columns in original dataset: 63
Columns remained after drop: 12

Dropping columns with NaN removed 80.95 % of the data!
Shape of Dataframe after columns manipulation: (21477, 12)
Dtype, Unique values and Missing(%) of each columns:

                     Dtype  Unique  Missing%
Area Abbreviation   object     169      0.00
Area Code            int64     174      0.00
Area                object     174      0.00
Item Code            int64     117      0.00
Item                object     115      0.00
...                    ...     ...       ...
Y2009              float64    2029      0.48
Y2010              float64    2046      0.48
Y2011              float64    2081      0.48
Y2012        

,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,1928.0,1904.0,1666.0,1950.0,2001.0,1808.0,2053.0,2045.0,2154.0,1819.0,1963.0,2215.0,2310.0,2335.0,2434.0,2512.0,2282.0,2454.0,2443.0,2129.0,2133.0,2068.0,1994.0,1851.0,1791.0,1683.0,2194.0,1801.0,1754.0,1640.0,1539.0,1582.0,1840.0,1855.0,1853.0,2177.0,2343.0,2407.0,2463.0,2600.0,2668.0,2776.0,3095.0,3249.0,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895
1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,183.0,183.0,182.0,220.0,220.0,195.0,231.0,235.0,238.0,213.0,205.0,233.0,246.0,246.0,255.0,263.0,235.0,254.0,270.0,259.0,248.0,217.0,217.0,197.0,186.0,200.0,193.0,202.0,191.0,199.0,197.0,249.0,218.0,260.0,319.0,254.0,326.0,347.0,270.0,372.0,411.0,448.0,460.0,419.0,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422
2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,76.0,76.0,76.0,76.0,76.0,75.0,71.0,72.0,73.0,74.0,71.0,70.0,72.0,76.0,77.0,80.0,60.0,65.0,64.0,64.0,60.0,55.0,53.0,51.0,48.0,46.0,46.0,47.0,46.0,43.0,43.0,40.0,50.0,46.0,41.0,44.0,50.0,48.0,43.0,26.0,29.0,70.0,48.0,58.0,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360
3,AFG,2,Afghanistan,2513,Barley and products,5142,Food,1000 tonnes,33.94,67.71,237.0,237.0,237.0,238.0,238.0,237.0,225.0,227.0,230.0,234.0,223.0,219.0,225.0,240.0,244.0,255.0,185.0,203.0,198.0,202.0,189.0,174.0,167.0,160.0,151.0,145.0,145.0,148.0,145.0,135.0,132.0,120.0,155.0,143.0,125.0,138.0,159.0,154.0,141.0,84.0,83.0,122.0,144.0,185.0,43.0,44.0,48.0,62.0,55.0,60.0,72.0,78,89
4,AFG,2,Afghanistan,2514,Maize and products,5521,Feed,1000 tonnes,33.94,67.71,210.0,210.0,214.0,216.0,216.0,216.0,235.0,232.0,236.0,200.0,201.0,216.0,228.0,231.0,234.0,240.0,228.0,234.0,228.0,226.0,210.0,199.0,192.0,182.0,173.0,170.0,154.0,148.0,137.0,144.0,126.0,90.0,141.0,150.0,159.0,108.0,90.0,99.0,72.0,35.0,48.0,89.0,63.0,120.0,208.0,233.0,249.0,247.0,195.0,178.0,191.0,200,200


In [8]:
# sposta nella sezione di data cleaning
old_keys = [n for n in df.columns if 'Y' in n]
new_keys = [n[1:] for n in old_keys]

# for n in range(len(old_keys)):
#     df.rename(columns={old_keys[n]: new_keys[n]}, inplace=True)

NameError: name 'df' is not defined